第32回 multiprocessingでの並列処理

時間がかかるループを使う場合は使える形に無理やりでもして，並列処理しよう 笑

最初は本記事(🐢#32)に書いてあるコードを毎回コピペして使える程度でOK

In [1]:
from multiprocessing import Pool

# Pool instance
cpu_num = 4
p = Pool(processes=cpu_num)

物理的に存在するコア → 物理コア　　　　　　　　　　　　　　　　　　　　　　　　　　
Hyper Threadingを使った擬似的なコア → 論理コア　　　　　　それぞれ確認コマンド有

In [2]:
# multiprpcessingでもコマンド有り
from multiprocessing import cpu_count
cpu_count()

8

上を返り値にして動的に　ただしCPUを占用するのは危険なので-1して指定

In [3]:
# 第30回を並列処理(p.map())で
import os
import numpy as np
import cv2
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from glob import glob

p = Pool(processes=cpu_count()-1)

save_base_path = 'gaussian_filtered_nparrays_multi/'

# folder作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)

def save_npy(path):
    
    # pngロード
    im = cv2.imread(path)
    
    # pngファイル名をそのまま.npyのファイル名にする
    file_name = path.split('/')[-1].split('.png')[0]
    save_path = save_base_path + file_name
    
    # 格納
    np.save(save_path, im)

# if __name__ == '__main__':
path_list = glob('gaussian_filtered_images/*/*.png')

list(tqdm(p.map(save_npy, path_list), total=len(path_list)))

# ★並列処理が終わったら閉じる
p.close()
p.join()

100%|██████████| 3662/3662 [00:00<00:00, 3720819.10it/s]


In [4]:
# 第30回を並列処理(p.imap())で
import os
import numpy as np
import cv2
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from glob import glob

p = Pool(processes=cpu_count()-1)

save_base_path = 'gaussian_filtered_nparrays_multi/'

# folder作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)

def save_npy(path):
    
    # pngロード
    im = cv2.imread(path)
    
    # pngファイル名をそのまま.npyのファイル名にする
    file_name = path.split('/')[-1].split('.png')[0]
    save_path = save_base_path + file_name
    
    # 格納
    np.save(save_path, im)

# if __name__ == '__main__':
path_list = glob('gaussian_filtered_images/*/*.png')

list(tqdm(p.imap(save_npy, path_list), total=len(path_list)))

# ★並列処理が終わったら閉じる
p.close()
p.join()

100%|██████████| 3662/3662 [00:09<00:00, 403.09it/s]


ほぼimap()のほうが有利なケースだと思うのでこっちを使って!

In [6]:
import os
import numpy as np
import cv2
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from glob import glob
 
p = Pool(processes=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays_multi_w_type/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
    
    
def wrap_save_npy(args):
    return save_npy(*args)
    
    
def save_npy(path, type_name):
    
    #pngデータをopencvでロード
    im = cv2.imread(path)
    #pngのファイル名 + フォルダ名を.npyファイルのファイル名にする
    file_name = path.split('/')[-1].split('.png')[0]
    save_path = save_base_path + file_name + '_' + type_name
    #全てのnpyファイルを同じフォルダに格納
    np.save(save_path, im)
    
path_list = glob('gaussian_filtered_images/*/*.png')
job_args = [(path, path.split('/')[1]) for path in path_list]
    
list(tqdm(p.imap(wrap_save_npy, job_args), total=len(path_list)))
p.close()
p.join()


100%|██████████| 3662/3662 [00:09<00:00, 376.32it/s]
